In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

df1E = pd.read_csv("/content/drive/My Drive/Colab Notebooks/df1E.csv") # df1E comes from 'Main' notebook
df1E.drop('Unnamed: 0', axis=1, inplace=True)
df1E

,collision_no,acc_date,year,month_name,day_name,acctime,longitude,latitude,X,Y,road_class,road_surface_cond,visibility,light,involved_class,involved_injury_class,# involved,# injured,# fatalities,# KSI
0,1,2000-09-11,2000,September,Monday,850,-79.456317,43.727732,-8.845037e+06,5.423404e+06,Minor Arterial,DRY,CLEAR,DAYLIGHT,"DRIVER, DRIVER","NONE, NONE",2,0,0,0
1,2,2000-09-11,2000,September,Monday,850,-79.456271,43.727698,-8.845032e+06,5.423399e+06,Minor Arterial,DRY,CLEAR,DAYLIGHT,"PASSENGER, PASSENGER, VEHICLE OWNER, VEHICLE O...","MINIMAL, MINIMAL, NONE, NONE",4,2,0,0
2,3,2000-01-02,2000,January,Sunday,630,-79.398357,43.706721,-8.838585e+06,5.420168e+06,Major Arterial,WET,RAIN,DARK,"DRIVER, VEHICLE OWNER, DRIVER","NONE, NONE, NONE",3,0,0,0
3,4,2000-01-02,2000,January,Sunday,1100,-79.395091,43.691344,-8.838221e+06,5.417801e+06,Major Arterial,WET,RAIN,DAYLIGHT,"DRIVER, DRIVER","NONE, NONE",2,0,0,0
4,5,2000-01-02,2000,January,Sunday,1125,-79.370090,43.656146,-8.835438e+06,5.412383e+06,Minor Arterial,WET,RAIN,DAYLIGHT,"VEHICLE OWNER, DRIVER, VEHICLE OWNER, DRIVER","NONE, NONE, NONE, NONE",4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990856,1038394,2019-02-24,2019,February,Sunday,1845,-79.319599,43.768005,-8.829817e+06,5.429610e+06,Major Arterial,LOOSE SNOW,SNOW,"DARK, ARTIFICIAL","DRIVER, PASSENGER, DRIVER, PASSENGER, PASSENGER","NONE, NONE, NONE, NONE, MINIMAL",5,1,0,0
990857,1038395,2019-10-20,2019,October,Sunday,1437,-79.587139,43.668288,-8.859600e+06,5.414252e+06,Local,DRY,CLEAR,DAYLIGHT,"DRIVER, DRIVER","NONE, NONE",2,0,0,0
990858,1038396,2019-05-10,2019,May,Friday,2200,-79.351940,43.677554,-8.833418e+06,5.415678e+06,Major Arterial,DRY,CLEAR,"DARK, ARTIFICIAL","DRIVER, DRIVER, DRIVER, PASSENGER","MAJOR, NONE, NONE, MINOR",4,2,0,1
990859,1038397,2019-07-20,2019,July,Saturday,2236,-79.314602,43.678378,-8.829261e+06,5.415805e+06,Minor Arterial,WET,RAIN,"DARK, ARTIFICIAL",DRIVER,NONE,1,0,0,0


In [0]:
# DBSCAN parameters
eps_in_meters = 200.0  # minimum distance between cluster points (eps)
minPts = 240           # minimum number of points per cluster (minPts)

In [0]:
# Cluster the data
from sklearn.cluster import DBSCAN
import math

earth_perimeter = 40070000.0  # in meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

# create a new dataframe
df1E_cl = df1E.copy()
# df1E_cl = df1E[['latitude', 'longitude', '# KSI']]
df1E_cl['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=minPts, metric='haversine').fit_predict(df1E_cl[['latitude', 'longitude']]) 
                                                                    # haversine distance (i.e. longitude/latitude distances)  
                                                               
df1E_cl

,collision_no,acc_date,year,month_name,day_name,acctime,longitude,latitude,X,Y,road_class,road_surface_cond,visibility,light,involved_class,involved_injury_class,# involved,# injured,# fatalities,# KSI,cluster
0,1,2000-09-11,2000,September,Monday,850,-79.456317,43.727732,-8.845037e+06,5.423404e+06,Minor Arterial,DRY,CLEAR,DAYLIGHT,"DRIVER, DRIVER","NONE, NONE",2,0,0,0,0
1,2,2000-09-11,2000,September,Monday,850,-79.456271,43.727698,-8.845032e+06,5.423399e+06,Minor Arterial,DRY,CLEAR,DAYLIGHT,"PASSENGER, PASSENGER, VEHICLE OWNER, VEHICLE O...","MINIMAL, MINIMAL, NONE, NONE",4,2,0,0,-1
2,3,2000-01-02,2000,January,Sunday,630,-79.398357,43.706721,-8.838585e+06,5.420168e+06,Major Arterial,WET,RAIN,DARK,"DRIVER, VEHICLE OWNER, DRIVER","NONE, NONE, NONE",3,0,0,0,-1
3,4,2000-01-02,2000,January,Sunday,1100,-79.395091,43.691344,-8.838221e+06,5.417801e+06,Major Arterial,WET,RAIN,DAYLIGHT,"DRIVER, DRIVER","NONE, NONE",2,0,0,0,-1
4,5,2000-01-02,2000,January,Sunday,1125,-79.370090,43.656146,-8.835438e+06,5.412383e+06,Minor Arterial,WET,RAIN,DAYLIGHT,"VEHICLE OWNER, DRIVER, VEHICLE OWNER, DRIVER","NONE, NONE, NONE, NONE",4,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990856,1038394,2019-02-24,2019,February,Sunday,1845,-79.319599,43.768005,-8.829817e+06,5.429610e+06,Major Arterial,LOOSE SNOW,SNOW,"DARK, ARTIFICIAL","DRIVER, PASSENGER, DRIVER, PASSENGER, PASSENGER","NONE, NONE, NONE, NONE, MINIMAL",5,1,0,0,-1
990857,1038395,2019-10-20,2019,October,Sunday,1437,-79.587139,43.668288,-8.859600e+06,5.414252e+06,Local,DRY,CLEAR,DAYLIGHT,"DRIVER, DRIVER","NONE, NONE",2,0,0,0,-1
990858,1038396,2019-05-10,2019,May,Friday,2200,-79.351940,43.677554,-8.833418e+06,5.415678e+06,Major Arterial,DRY,CLEAR,"DARK, ARTIFICIAL","DRIVER, DRIVER, DRIVER, PASSENGER","MAJOR, NONE, NONE, MINOR",4,2,0,1,-1
990859,1038397,2019-07-20,2019,July,Saturday,2236,-79.314602,43.678378,-8.829261e+06,5.415805e+06,Minor Arterial,WET,RAIN,"DARK, ARTIFICIAL",DRIVER,NONE,1,0,0,0,-1


In [0]:
df1E_cl['cluster'].nunique()

344

In [0]:
# Noise points are marked with cluster number -1 and excluded from our analysis.
df1E_cl[df1E_cl['cluster']==-1].shape[0]

856323

In [0]:
print('Out of the total {:,} KSI, {:,} are detected as noise and are not in the clusters.'.format(df1E_cl['# KSI'].sum(), df1E_cl[df1E_cl['cluster']==-1]['# KSI'].sum())) 

Out of the total 9,814 KSI, 8,613 are detected as noise and are not in the clusters.


In [0]:
!pip install geopandas
import geopandas as gpd
from shapely.geometry import Point, Polygon

     |████████████████████████████████| 921kB 2.8MB/s 
     |████████████████████████████████| 10.1MB 47.9MB/s 
     |████████████████████████████████| 11.8MB 41.2MB/s 


In [0]:
ksi = df1E_cl[df1E_cl['cluster']==100]['# KSI'].sum()
ksi

In [0]:
ksi_2 = df1E_cl.loc[df1E_cl['cluster'] == 100, '# KSI'].sum()
ksi_2

In [0]:
# Create a new GeoDataFrame to form the polygons that contain all the accident points in each cluster
crs = {'init': 'epsg:4326'}
cluster_polygon = gpd.GeoDataFrame(columns=['cluster', '# accidents', '# KSI', 'geometry'])

for i in range(df1E_cl['cluster'].nunique()):
    ply = Polygon(zip(df1E_cl[df1E_cl['cluster']==i]['longitude'], df1E_cl[df1E_cl['cluster']==i]['latitude']))  
    cluster_polygon.loc[i, 'geometry'] = ply
    
    num_acc = df1E_cl[df1E_cl['cluster']==i].shape[0]
    cluster_polygon.loc[i, '# accidents'] = num_acc
    
    ksi = df1E_cl[df1E_cl['cluster']==i]['# KSI'].sum()
    cluster_polygon.loc[i, '# KSI'] = ksi
    
    cluster_polygon.loc[i, 'cluster'] = i
    
cluster_polygon[['cluster', '# accidents', '# KSI']] = cluster_polygon[['cluster', '# accidents', '# KSI']].astype(np.int16)
cluster_polygon

,cluster,# accidents,# KSI,geometry
0,0,286,0,"POLYGON ((-79.456 43.728, -79.456 43.728, -79...."
1,1,390,1,"POLYGON ((-79.268 43.788, -79.268 43.788, -79...."
2,2,536,14,"POLYGON ((-79.355 43.793, -79.355 43.793, -79...."
3,3,317,0,"POLYGON ((-79.580 43.740, -79.580 43.740, -79...."
4,4,388,4,"POLYGON ((-79.370 43.646, -79.370 43.646, -79...."
...,...,...,...,...
339,339,253,4,"POLYGON ((-79.445 43.781, -79.445 43.781, -79...."
340,340,262,1,"POLYGON ((-79.410 43.758, -79.410 43.758, -79...."
341,341,418,2,"POLYGON ((-79.394 43.669, -79.394 43.669, -79...."
342,342,241,1,"POLYGON ((-79.328 43.659, -79.328 43.659, -79...."


In [0]:
# Maximum number of KSI and their clusters
cluster_polygon[cluster_polygon['# KSI'] == cluster_polygon['# KSI'].max()]

,cluster,# accidents,# KSI,geometry
12,12,450,15,"POLYGON ((-79.56861 43.74624, -79.56861 43.746..."
36,36,453,15,"POLYGON ((-79.26129 43.77005, -79.26132 43.770..."


In [0]:
# Maximum number of accidents and their clusters
cluster_polygon[cluster_polygon['# accidents'] == cluster_polygon['# accidents'].max()]

,cluster,# accidents,# KSI,geometry
293,293,1157,0,"POLYGON ((-79.55124 43.75652, -79.55124 43.756..."


In [0]:
cluster_polygon.to_csv('df1E_cluster_polygon.csv')

In [0]:
# cluster_polygon_2 = cluster_polygon.copy()
# cluster_polygon_2['geometry'] = cluster_polygon_2['geometry'].convex_hull
# cluster_polygon_2.to_csv('df1E_cluster_polygon_2.csv')

In [0]:
# acc_points = gpd.GeoSeries([Point(x, y) for x, y in zip(df1E_cl['longitude'], df1E_cl['latitude'])])
# circles = acc_points.buffer(2000)
# mp = circles.unary_union
# holes = cluster_polygon['geometry'].intersection(mp)